In [3]:
import numpy as np
import pandas as pd
import re
from datetime import datetime as dt

In [4]:
url = 'https://raw.githubusercontent.com/hkfuoc98/Imaginary-Data-Projects/master/Case%20Ecommerce%20Logistics.csv'
df = pd.read_csv(url)
df['time'] = pd.to_datetime(df['time'],dayfirst = True,utc=True)
df.head()

,order number,time,event
0,#1,2022-07-14 14:41:00+00:00,Giao hàng thành công. Người nhận hàng: XXXXX
1,#1,2022-07-14 14:35:00+00:00,Đang giao hàng
2,#1,2022-07-14 12:39:00+00:00,Giao hàng không thành công vì Người nhận hẹn l...
3,#1,2022-07-14 08:57:00+00:00,Đang giao hàng
4,#1,2022-07-14 05:23:00+00:00,Đơn hàng đã đến kho 50-HCM Quan 10/Phuong 15 L...


In [63]:
# Starting: đã đến kho Cu Chi SOC
# First dilivery: 'Giao hàng%''

from_cu_chi = df[df['event']=='Đơn hàng đã đến kho Cu Chi SOC'].loc[:,['order number','time']]
from_cu_chi.head()

,order number,time
6,#1,2022-07-13 22:22:00+00:00
19,#2,2022-05-05 09:46:00+00:00
30,#3,2022-02-17 14:21:00+00:00
39,#4,2022-01-22 00:08:00+00:00


In [7]:
first_delivery = df[df['event'].str.startswith('Giao hàng')].loc[:,['order number','time']]
first_delivery = first_delivery.groupby(by=['order number'])['time'].min()
first_delivery.head()

order number
#1   2022-07-14 12:39:00+00:00
#2   2022-05-06 18:17:00+00:00
#3   2022-02-18 10:10:00+00:00
#4   2022-01-22 15:51:00+00:00
Name: time, dtype: datetime64[ns, UTC]

In [68]:
first_delivery.columns = ('order number','first delivery')
to_first_deli = from_cu_chi.set_index('order number').join(first_delivery.set_index('order number'))
to_first_deli['hrs taken']=to_first_deli['first delivery']-to_first_deli['time']
mean = to_first_deli['hrs taken'].mean()
med = to_first_deli['hrs taken'].median()
print('mean of 1st time '+ str(mean))
print('median of 1st time '+ str(med))

mean of 1st time 0 days 20:35:00
median of 1st time 0 days 17:46:00


In [71]:
# Lọc các đơn không thông qua hub: 50-HCM Quan 10/Phuong 15 LM Hub
df[df['event'].str.contains('50-HCM Quan 10/Phuong 15 LM Hub')]

,order number,time,event
4,#1,2022-07-14 05:23:00+00:00,Đơn hàng đã đến kho 50-HCM Quan 10/Phuong 15 L...
17,#2,2022-05-06 12:04:00+00:00,Đơn hàng đã đến kho 50-HCM Quan 10/Phuong 15 L...


In [98]:
effective = df[df['event'].str.contains('50-HCM Quan 10/Phuong 15 LM Hub')].set_index('order number')
df[~df['order number'].isin(effective.index)]

,order number,time,event
26,#3,2022-02-18 10:10:00+00:00,Giao hàng thành công. Người nhận hàng: XXX
27,#3,2022-02-18 07:35:00+00:00,Đang giao hàng
28,#3,2022-02-17 20:55:00+00:00,Đơn hàng đã đến kho 50-HCM Phu Nhuan/Phuong 4 ...
29,#3,2022-02-17 18:16:00+00:00,Đơn hàng đã xuất kho
30,#3,2022-02-17 14:21:00+00:00,Đơn hàng đã đến kho Cu Chi SOC
31,#3,2022-02-17 13:24:00+00:00,Đơn hàng đã xuất kho
32,#3,2022-02-17 12:44:00+00:00,Đơn hàng đã đến kho 50-HCM Tan Phu/Hoa Thanh SOC
33,#3,2022-02-17 11:39:00+00:00,Lấy hàng thành công
34,#3,2022-02-17 08:13:00+00:00,Người gửi đang chuẩn bị hàng
35,#4,2022-01-22 15:51:00+00:00,Giao hàng thành công. Người nhận hàng: XXXXX
